# Object Detection on Custom Data. 😎️

**#Just GPU Things**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

**Accessing Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**List of items in your "gdrive"**

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



**Let's go Dark.**
As we will clone the darknet you may not be able to see it in your drive page, but you can explore it using linux commands(ie. cd / ls etc).

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**2) Now let's set the darknet with GPU.**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

We will copy the "yolov3-tiny.cfg" file from cfg directory to "yolov3_training.cfg".

In [ ]:
!cp cfg/yolov3-tiny.cfg cfg/yolov3_training.cfg

**Let's edit some configurations of the cfg file.**(pun)

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

**Let's create a folder named "yolov3_w1" to save the new weights.** (trash name, I know)

In [ ]:
#@title
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3_w1"

**We will make a file called "obj.data" in data directory that will keep all the classes names, here I have only used one class, so bear with me 😌️**

In [ ]:
!echo "Bucket" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3_w1' > data/obj.data
!mkdir data/obj

**For training we need a weight file to train and save it, let's use "darknet53.conv.74"**

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

For that you have to upload the data(including image and label file) on drive.
Now you have to edit the path in the given code.

In [ ]:
!unzip /mydrive/Falcon_Computer_Vision/new_dataset.zip -d data/obj

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/new_dataset/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/new_dataset/*.jpg")

## Perform Normalization!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
### I have not done it........

print(images_list)

In [ ]:
#Create train.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

**Be patient and wait for it to converge, now what is "converge" please Google in your waiting time and utilize your time, atleast for now. 🙂️**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show